In [8]:
%matplotlib inline
from krazy_gridworld import KrazyGridWorld
import gym 
import numpy as np
import torch
from model import Model
from utils import ReplayBuffer, get_state, advice_satisfied
from itertools import count
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

## Hyperparameters

In [9]:
NAME='Single-Goal-v2' + ' ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

max_frames = int(32e6)
save_interval = 100

lr=1e-3
batch_size = 64
epsilon = 0.05
T=25

log_dir = f'/mnt/hdd1/ml/logs/{NAME}'
SAVE_DIR = 'models'

In [10]:
env = KrazyGridWorld()
env.reset()
channel_in, height, width = get_state(env).shape
action_dim = 4

In [11]:
writer = SummaryWriter(log_dir=log_dir)
# writer=None
replay_buffer = ReplayBuffer()

In [12]:
net = Model(lr, channel_in, height, width, action_dim, writer=writer)

## Utility functions

In [13]:
def epsilon_decay(frame_number, eps_init=1.0, eps_end=0.01, decay_len=50000):
    if frame_number > decay_len:
        return eps_end
    else:
        return eps_init * (1-frame_number/decay_len) + eps_end * (frame_number/decay_len)

## Training

In [14]:
frame_number = 0
dqn_num = 0
success = 0
num_episodes = 0
while frame_number < max_frames:
    env.reset()
    satisfied = False
    state = get_state(env)
    
    for t in range(25):
        frame_number += 1
        state = get_state(env)
        eps = epsilon_decay(frame_number)
        action = net.select_action(state, dqn_num, epsilon=eps)
        _, _, done, info = env.step(action)
        next_state = get_state(env)
        at_goal = env.at_goal()
        is_lava = env.is_dead()
        color = info['color']
        
        if advice_satisfied(color, at_goal, is_lava):
            reward = 1
            satisfied = True
        else:
            reward = 0
        replay_buffer.add(state, action, reward, next_state, float(done))        
            
        if done or t == 24:
            if satisfied:
                success += 1
                replay_buffer.set_one(t)
            num_episodes += 1
            break  
            
    loss = net.update(batch_size, replay_buffer, (dqn_num + 1)% 2)
    if writer is not None and loss is not None:
        writer.add_scalar("loss", loss, frame_number)
    
    if writer is not None and num_episodes > 100:
        writer.add_scalar('success_rate', success/num_episodes, frame_number)
        success = 0
        num_episodes = 0
        
    dqn_num = (dqn_num + 1) % 2
        
#     if episode % save_interval == 0:
#         print(f'model saved on episode: {episode % (10 * save_interval)}')
#         net.save('models', f'episode-{episode % (10 * save_interval)}')
        
#         print(f'best model saved with reward: {total_rewards}')
#         net.save('models', f'best')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tony/anaconda3/envs/ml/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e5975021b0ec>", line 14, in <module>
    action = net.select_action(state, dqn_num, epsilon=eps)
  File "/mnt/hdd1/ml/ACTRCE-Experiments/Single-Goal-v2/model.py", line 67, in select_action
    return torch.argmax(action).cpu().item()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tony/anaconda3/envs/ml/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tony/anaconda3/envs/ml/lib/python3.7/sit

KeyboardInterrupt: 

In [ ]:
# net.load('models', f'best')

In [ ]:
# def show_state(env, step=0, info=""):
#     plt.figure(3)
#     plt.clf()
#     plt.imshow(env.get_img_pyplot_obs())
#     plt.title("%s | Step: %d %s" % (env, step, info))
#     plt.axis('off')

#     display.clear_output(wait=True)
#     display.display(plt.gcf())